In [1]:
import import_ipynb
from db_setup import get_collections
from tabulate import tabulate
from datetime import datetime

collections = get_collections()
loan_requests_collection = collections["loan_requests"]
loans_collection = collections["loans"]
lender_profiles_collection = collections["lender_profiles"]
borrower_profiles_collection = collections["borrower_profiles"]
users_collection = collections["users"]

# Method to print all the available loan requests
def print_all_loan_requests():
    loan_requests = list(loan_requests_collection.find({"status": "pending"}))
    
    if not loan_requests:
        print("No loan requests available to invest.")
        return
    
    table_data = []
    for request in loan_requests:
        table_data.append([
            request.get("loan_request_id", "N/A"),
            request.get("borrower_id", "N/A"),
            request.get("requested_amount", "N/A"),
            request.get("predicted_interest_rate", "0.0"),
            request.get("loan_term", "N/A"),
            request.get("homeownership", "N/A"),
            request.get("verified_income", "N/A"),
            request.get("purpose", "N/A"),
            request.get("grade", "N/A"),
            request.get("sub_grade", "N/A"),
            request.get("status", "N/A"),
            request.get("requested_date", "N/A"),
        ])
    
    headers = [
        "Request ID", "Borrower ID", "Requested Amount", "Interest Rate",
        "Loan Term", "Homeownership", "Verified Income",
        "Purpose", "Grade", "Sub Grade", "Status", "Requested Date"
    ]
    
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Method to invest on a loan request
def invest_in_loan(user):
    if user['type'] != 'lender':
        print("Only lenders can invest in loans.")
        return
    print_all_loan_requests()
    
    loan_request_id = input("Enter Loan Request ID to invest in: ")

    loan_request = loan_requests_collection.find_one({"loan_request_id": loan_request_id, "status": "pending"})
    predicted_interest_rate= loan_request.get("predicted_interest_rate",0)
    print("Do you want to lend with the predicted interest:",predicted_interest_rate)
    print("1. Yes")
    print("2. No")
    choice = input("Enter your choice: ")
    if choice == "1":
        interest_rate = predicted_interest_rate
    elif choice == "2":
        interest_rate = round(float(input("Enter the interest rate:")),2)
    lender_id = user['user_id']
    borrower_id = loan_request["borrower_id"]
    
    
    if not loan_request:
        print("Loan request not found or already approved.")
        return
    
    new_loan = {
        "loan_id": loan_request_id,
        "borrower_id": loan_request["borrower_id"],
        "lender_id": lender_id,
        "loan_amount": loan_request["requested_amount"],
        "interest_rate": interest_rate,
        "predicted_interest_rate": loan_request.get("predicted_interest_rate",0),
        "loan_term": loan_request["loan_term"],
        "homeownership": loan_request["homeownership"],
        "verified_income": loan_request["verified_income"],
        "purpose": loan_request["purpose"],
        "grade": loan_request['grade'],
        "sub_grade": loan_request['sub_grade'],
        "has_co_signer": loan_request['has_co_signer'],
        "has_dependants": loan_request['has_dependants'],
        "status": "active",
        
    }

    # Insert the approved loan request into the loans collection
    loans_collection.insert_one(new_loan)

    # Update the loan requests collection for that particular loan by updating its status
    loan_requests_collection.update_one({"loan_request_id": loan_request_id}, {"$set": {"status": "approved"}})

    # Updating the lender profiles to include this loan deatils into their portfolio
    lender_profile = lender_profiles_collection.find_one({"lender_id": lender_id})
    if lender_profile:
        lender_profiles_collection.update_one(
            {"lender_id": lender_id},
            {
                "$push": {"portfolio": {"loan_id": new_loan["loan_id"], "amount": new_loan["loan_amount"]}},
                "$inc": {"total_invested": new_loan["loan_amount"]}
            }
        )
    else:
        lender_profiles_collection.insert_one({
            "lender_id": lender_id,
            "income": user['annual_income'],
            "portfolio": [{"loan_id": new_loan["loan_id"], "amount": new_loan["loan_amount"]}],
            "total_invested": new_loan["loan_amount"]
        })


    # Updating the borrower profiles to include this loan deatils into their portfolio
    borrower_profile = borrower_profiles_collection.find_one({"borrower_id": borrower_id})
    borrower = users_collection.find_one({"user_id": borrower_id})
    if borrower_profile:
        updated_total_loan_amount = borrower_profile["total_loan_amount"] + loan_request["requested_amount"]
        debt_to_income_ratio = updated_total_loan_amount / borrower_profile["income"]
        borrower_profiles_collection.update_one(
            {"borrower_id": borrower_id},
            {
                "$push": {
                    "portfolio": {"loan_id": loan_request_id, "amount": loan_request["requested_amount"]}
                },
                "$inc": {
                    "total_loan_amount": loan_request["requested_amount"],
                    "num_credit_lines": 1
                },
                "$set": {
                    "debt_to_income_ratio": debt_to_income_ratio
                }
            }
        )
    else:
        borrower_profiles_collection.insert_one({
            "borrower_id": borrower_id,
            "income":  borrower['annual_income'],
            "portfolio": [{"loan_id": loan_request_id, "amount": loan_request["requested_amount"]}],
            "total_loan_amount": loan_request["requested_amount"],
            "debt_to_income_ratio": total_loan_amount / income if income > 0 else 0
        })
    
    print(f"Loan request {loan_request_id} successfully funded by lender {lender_id}.")

# Method to track investments made by a lender
def track_investments(user):

    lender_profile = lender_profiles_collection.find_one({"lender_id": user['user_id']})
    
    if not lender_profile:
        print(f"No investments found for Lender ID: {user['user_id']}")
        return
    
    loans = list(loans_collection.find({"lender_id": user['user_id']}))
    
    if not loans:
        print(f"No loans found for Lender ID: {user['user_id']}")
        return
    
    total_invested = lender_profile.get("total_invested", 0)

    print(f"Lender ID: {user['user_id']}")
    print(f"Total Invested Amount: ${total_invested:.2f}")
    print("\n--- Loan Details ---")
    for loan in loans:
        print(f"Loan ID: {loan['loan_id']}")
        print(f"Borrower ID: {loan['borrower_id']}")
        print(f"Loan Amount: ${loan['loan_amount']:.2f}")
        print(f"Interest Rate: {loan['interest_rate']}%")
        print(f"Loan Term: {loan['loan_term']} months")
        print(f"Status: {loan['status']}")
        print("-" * 40)